In [1]:
import geopy
import geopandas as gpd
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from halo import Halo
import folium
from folium.map import *
from folium.plugins import MiniMap, Search, BeautifyIcon
from folium.features import GeoJson, GeoJsonTooltip, CustomIcon
import time
import json
import requests
import branca

In [2]:
data_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data'
data_request = f'{data_url}/owid-covid-data.csv'
data = pd.read_csv(data_request,usecols=['location','total_cases','total_deaths'])

In [3]:
grouped_data = data.groupby('location').max().reset_index()

In [4]:
name_list = []
cases_list = []
deaths_list = []
for row in grouped_data.values:
    if row[0] == 'World' or row[0] == 'International':
        pass
    elif row[0] == 'United States':
        name = row[0].replace('United States','United States of America')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
        
    elif row[0] == 'Democratic Republic of Congo':
        name = row[0].replace('Democratic Republic of Congo','Democratic Republic of the Congo')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
    elif row[0] == 'Tanzania':
        name = row[0].replace('Tanzania','United Republic of Tanzania')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
    elif row[0] == 'Congo':
        name = row[0].replace('Congo','Republic of the Congo')
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])
    else:
        name = row[0]
        name_list.append(name)
        cases_list.append(row[1])
        deaths_list.append(row[2])



In [5]:
new_data = pd.DataFrame(list(zip(name_list,cases_list,deaths_list)), columns=['name','cases','deaths'])

In [6]:
geo_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
geo_data = f'{geo_url}/world-countries.json'
geo_json_data = json.loads(requests.get(geo_data).text)

In [7]:
states = gpd.GeoDataFrame.from_features(geo_json_data,crs='EPSG:4326')

In [8]:
statesmerge = states.merge(new_data, how='left', left_on='name', right_on='name')
statesmerge['geometry'] = statesmerge.geometry.simplify(.05)

In [9]:
mymap = folium.Map(location=[0, 0], tiles="OpenStreetMap",zoom_start=2,max_bounds=True,no_wrap=True)

In [10]:
colormap = branca.colormap.LinearColormap(
    vmin=statesmerge['cases'].min(), 
    vmax=statesmerge['cases'].max(),
    colors=['#FF0000','#FF3300','#ff6600','#ff9900','#FFCC00','#FFFF00','#ccff00','#99ff00','#66ff00','#33ff00','#00FF00'],
    caption="COVID-19 Cases around the world",
)

In [11]:
tooltip = GeoJsonTooltip(
    fields=["name", "cases", "deaths"],
    aliases=["Country:", "COVID-19 Cases:", "COVID-19 Deaths:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid red;
        border-radius: 3px;
        box-shadow: 3px;
    """,

)


g = folium.GeoJson(
    statesmerge,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["cases"])
        if x["properties"]["cases"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,

).add_to(mymap)

colormap.add_to(mymap)

In [12]:
statesearch = Search(
            layer=g,
            geom_type='Polygon',
            placeholder='Search for Country',
            collapsed=True,
            search_label='name',
            weight=3,
            position='bottomright',
        ).add_to(mymap)

            #mymap.add_child(folium.LatLngPopup())
            #mymap.add_child(folium.ClickForMarker(popup='Waypoint'))
myminimap = MiniMap(toggle_display=True,tile_layer='OpenStreetMap',position='bottomleft',minimized=True)
myminimap.add_to(mymap)


In [13]:
mymap

In [14]:
mymap.save('COVD-19-Cases.html')